In [75]:
import numpy as np

def load_glove_embeddings(glove_file_path):
    embeddings = {}
    with open(glove_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try: 
                values = line.split()
                word = values[0]
                vector = np.array(values[1:], dtype='float32')
                embeddings[word] = vector
            except:
                print('error')
    return embeddings

# Load the 100-dimensional GloVe embeddings
glove_file = './glove.840B.300d.txt'
glove_embeddings = load_glove_embeddings(glove_file)

# Check the embedding for a specific word
word = 'king'
print(f'Embedding for "{word}":\n', glove_embeddings.get(word))


error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
Embedding for "king":
 [ 3.1542e-01 -3.5068e-01  4.2923e-01 -5.3825e-01 -1.8480e-01 -3.1082e-01
  2.9196e-01 -7.1030e-01 -2.3867e-01  1.8471e+00 -3.6446e-01 -5.1282e-01
  1.2210e-01  3.8909e-01 -7.3204e-02  3.5462e-02  3.3289e-01  6.6466e-01
  2.7175e-02  4.2021e-01 -1.4520e-01  3.7991e-01 -6.0520e-01  1.0695e-01
 -6.4716e-01 -1.0739e-02 -3.9754e-01  3.8857e-01 -2.0134e-01  6.9813e-01
 -3.2411e-01  7.3085e-01 -1.0930e-01 -2.3511e-01  1.8482e-01 -1.1595e-01
 -7.1003e-01 -2.2974e-01 -4.1979e-01  8.1004e-03 -1.0504e-01 -4.4802e-01
 -7.3928e-02 -4.2380e-01  2.8482e-01 -7.4517e-02  9.8161e-02  6.4602e-01
 -2.5832e-01 -2.0452e-02 -6.6863e-02  5.1501e-01  1.6758e-01  1.2329e-01
  1.9636e-01  1.1958e-01 -1.8296e-01 -1.4325e-01 -2.7758e-01  5.0597e-02
 -6.6122e-02 -1.8920e-01  3.3300e-01  2.5319e-01  6.6355e-01  6.6735e-01
  4.9969e-01  1.5481e-01 -8.4247e-02 -2.2947e-01 -6.83

In [19]:
import sentence_transformers
import numpy as np
# load the pre-trained model
# model = sentence_transformers.SentenceTransformer('bert-base-nli-mean-tokens')

/home/oscar/.local/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [75]:
# Tangible,Intangible,Valuable,Worthless
# good, evil, new , old
# ax1 = ("simple", "complex")
ax1 = ("hot", "cold")
# ax2 = ("structured", "chaotic")
ax2 = ("dry", "wet")
xrel_key = get_embedding(ax1[0]) + get_embedding(ax1[1])
xdir_key = get_embedding(ax1[0]) - get_embedding(ax1[1])
xdir_key = xdir_key / np.linalg.norm(xdir_key)
yrel_key = get_embedding(ax2[0]) + get_embedding(ax2[1])
ydir_key = get_embedding(ax2[0]) - get_embedding(ax2[1])
ydir_key = ydir_key / np.linalg.norm(ydir_key)
def word_to_dir(word):
    vec = get_embedding(word)
    print(np.linalg.norm(vec))
    xrel = np.linalg.norm(np.dot(vec, xrel_key))
    xdir = np.dot(vec, xdir_key)
    yrel = np.linalg.norm(np.dot(vec, yrel_key))
    ydir = np.dot(vec, ydir_key)
    return xrel * xdir, yrel * ydir

word_to_dir('mathematics'), word_to_dir('contentment'), word_to_dir('rage'), word_to_dir('annoyance')

# pipeline
# for each axis
# - get relevance per word
# - get direction per word
# - normalize to std normal



0.99997216
1.000006
0.99999386
1.0000012


((-0.0008725859, -0.005724415),
 (-0.0050349664, -0.0015534267),
 (0.01916994, 0.001989362),
 (0.00021105981, -0.0011858245))

In [76]:
idx = model2.key_to_index['/c/en/weather']
# polnp.linalg.norm(model2.vectors[idx])
xrel_key = yrel_key = model2.vectors[idx]

In [77]:
xrels = np.abs(model2.vectors @ xrel_key)**2
xdirs = model2.vectors @ xdir_key * xrels
xdirs = (xdirs - np.mean(xdirs)) / np.std(xdirs)
# xscores = xdirs * xrels
# xscores = (xscores - np.mean(xscores))
# xscores = xscores / np.std(xscores)
# xscores
# yrels = np.abs(model2.vectors @ yrel_key)**2
ydirs = model2.vectors @ ydir_key * xrels
# yscores = ydirs * yrels
# yscores = (yscores - np.mean(yscores)) 
# yscores = yscores / np.std(yscores)
# yscores
ydirs = (ydirs - np.mean(ydirs)) / np.std(ydirs)
# stack dirs and normalize
dirs = np.stack((xdirs, ydirs), axis=1)
dirs = dirs / np.linalg.norm(dirs, axis=1, keepdims=True)
scores = dirs * (np.abs(model2.vectors @ xrel_key)**1)[:,None]




print(ax1, ax2)
# for word in ("reform", "dictatorship", "freedom", "tyranny", "revolution"):
for word in ("blizzard", "snowy", "desert"):
    print("word: ", word)
    idx = model2.key_to_index[f'/c/en/{word}']
    print("scores: ", scores[idx])
# n.b. I think that because I normalize I don't need to use xrels and yrels
# masculine/feminine assigns king as feminine , better just to use a simple axis like man/woman

# category: facial
# for word in ("maniacal")

# category: politics
# (good, evil) (wild, tame)
# for word in ("dog", "wolf", "werewolf", "freedom", "peace", "tyranny", "revolution"):


('hot', 'cold') ('dry', 'wet')
word:  blizzard
scores:  [-0.43811432 -0.31297714]
word:  snowy
scores:  [-0.29574075 -0.31410658]
word:  desert
scores:  [0.005318   0.13764565]


In [78]:
import wordfreq
# top 100
top_100k = wordfreq.top_n_list('en', 10**5)
top_100k.__len__()
out = {}
for word in top_100k:
    try:
        idx = model2.key_to_index[f'/c/en/{word}']
        xscore, yscore = scores[idx]
        out[word] = int(xscore * 1000.0)/1000.0, int(yscore * 1000.0)/1000.0
    except:
        # print(f"word {word} not found")
        pass
out["monster"]

(0.018, 0.004)

In [79]:
import json
# only 3 decimals of precision
file = open("g5.json", "w")
j = json.dump(out, file)
len(j)


TypeError: object of type 'NoneType' has no len()

In [237]:
j[:1000]

'{"the": [0.035, 0.11], "to": [0.001, 0.0], "and": [0.134, 0.097], "of": [0.008, 0.046], "a": [0.019, 0.007], "in": [0.266, 1.328], "i": [0.202, 0.025], "is": [0.646, 0.049], "for": [0.0, 0.0], "that": [0.138, 0.089], "you": [0.006, 0.005], "it": [0.0, 0.0], "on": [0.116, 0.385], "with": [0.0, 0.058], "this": [0.099, 0.062], "was": [0.018, 0.0], "be": [0.018, -0.004], "as": [0.174, -0.004], "are": [0.0, 0.0], "have": [0.144, -0.012], "at": [0.005, 0.001], "he": [0.002, 0.002], "not": [1.947, -0.316], "by": [0.007, 0.027], "but": [0.482, -0.238], "from": [0.0, 0.0], "my": [0.034, 0.011], "or": [0.119, 0.486], "we": [0.064, 0.049], "an": [0.0, 0.0], "your": [0.019, 0.024], "all": [0.028, 0.133], "so": [0.0, 0.0], "his": [-0.019, 0.083], "they": [0.089, 0.039], "me": [0.009, 0.001], "if": [0.1, 0.036], "one": [0.002, 0.003], "can": [0.007, 0.0], "will": [0.071, 0.049], "just": [0.704, -0.039], "like": [0.241, 0.025], "about": [0.13, 0.059], "up": [0.26, 0.041], "out": [0.01, 0.015], "what

In [21]:
import numpy as np
w1 = "dog"
w2 = "water"
np.dot(get_embedding(w1), get_embedding(w2)) 

0.048080556

In [4]:
def get_embedding(word):
    return model2[f"/c/en/{word}"]

In [2]:
# Load ConceptNet Numberbatch embeddings (300-dimensional vectors)
import gensim.downloader as api
model2 = api.load("conceptnet-numberbatch-17-06-300")


NameError: name 'model' is not defined

In [3]:
model2["/c/en/anthropocentric"]

array([-0.0831, -0.1472, -0.1256, -0.0556,  0.1531,  0.1012, -0.0885,
        0.0704,  0.046 ,  0.0744,  0.1335, -0.038 ,  0.0213, -0.0432,
        0.1401,  0.2077,  0.076 , -0.0194, -0.0143,  0.0101, -0.128 ,
       -0.0642,  0.0363,  0.1289, -0.0221, -0.0652, -0.0569, -0.0522,
       -0.0259,  0.0415,  0.0597, -0.0138, -0.0779,  0.1409, -0.0251,
       -0.0439,  0.0218,  0.1129,  0.0074,  0.0102, -0.0129,  0.0114,
        0.0295, -0.0374, -0.0339, -0.0563, -0.0354, -0.0005,  0.0027,
       -0.024 , -0.0533,  0.0511,  0.0809, -0.0045, -0.0284,  0.0167,
        0.056 , -0.0101, -0.0091,  0.0739,  0.0074,  0.0707, -0.0873,
       -0.0338,  0.075 ,  0.05  , -0.0404,  0.026 ,  0.0617,  0.0071,
       -0.01  ,  0.0452, -0.1058, -0.0298,  0.0631,  0.0463, -0.1158,
       -0.1217,  0.0917,  0.0172,  0.0171, -0.0064,  0.1632,  0.0642,
        0.0418, -0.0055, -0.0374,  0.004 ,  0.0315,  0.0014,  0.0588,
        0.0506,  0.0019, -0.037 ,  0.0823, -0.0206, -0.0767, -0.0211,
        0.0344, -0.0

In [15]:
model.key_to_index

{'/c/af/1_konings': 0,
 '/c/af/2_konings': 1,
 '/c/af/a.s': 2,
 '/c/af/a_foei_tog': 3,
 '/c/af/a_ja_a': 4,
 '/c/af/a_nee_a': 5,
 '/c/af/aag': 6,
 '/c/af/aai': 7,
 '/c/af/aak': 8,
 '/c/af/aaklig': 9,
 '/c/af/aakligheid': 10,
 '/c/af/aal': 11,
 '/c/af/aalmoes': 12,
 '/c/af/aalmoesenier': 13,
 '/c/af/aalmoeseniershuis': 14,
 '/c/af/aalmoesgewer': 15,
 '/c/af/aaltjie': 16,
 '/c/af/aalwee': 17,
 '/c/af/aalwurm': 18,
 '/c/af/aalwyn': 19,
 '/c/af/aambeeld': 20,
 '/c/af/aambeeldvoël': 21,
 '/c/af/aambei': 22,
 '/c/af/aamborstig': 23,
 '/c/af/aamborstigheid': 24,
 '/c/af/aan': 25,
 '/c/af/aanbehoort': 26,
 '/c/af/aanbel': 27,
 '/c/af/aanbestee': 28,
 '/c/af/aanbetref': 29,
 '/c/af/aanbetrou': 30,
 '/c/af/aanbeveel': 31,
 '/c/af/aanbeveelbaar': 32,
 '/c/af/aanbevelenswaardig': 33,
 '/c/af/aanbeveling': 34,
 '/c/af/aanbid': 35,
 '/c/af/aanbidbaar': 36,
 '/c/af/aanbidbaarheid': 37,
 '/c/af/aanbiddelik': 38,
 '/c/af/aanbiddelikheid': 39,
 '/c/af/aanbied': 40,
 '/c/af/aanbieder': 41,
 '/c/af/aanbied

In [2]:
import gensim.downloader as api

print(api.info())  # This will list available pre-trained models like Word2Vec, GloVe, FastText, etc.


{'corpora': {'semeval-2016-2017-task3-subtaskBC': {'num_records': -1, 'record_format': 'dict', 'file_size': 6344358, 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/semeval-2016-2017-task3-subtaskB-eng/__init__.py', 'license': 'All files released for the task are free for general research use', 'fields': {'2016-train': ['...'], '2016-dev': ['...'], '2017-test': ['...'], '2016-test': ['...']}, 'description': 'SemEval 2016 / 2017 Task 3 Subtask B and C datasets contain train+development (317 original questions, 3,169 related questions, and 31,690 comments), and test datasets in English. The description of the tasks and the collected data is given in sections 3 and 4.1 of the task paper http://alt.qcri.org/semeval2016/task3/data/uploads/semeval2016-task3-report.pdf linked in section “Papers” of https://github.com/RaRe-Technologies/gensim-data/issues/18.', 'checksum': '701ea67acd82e75f95e1d8e62fb0ad29', 'file_name': 'semeval-2016-2017-task3-subtaskBC.gz',